In [75]:
# imoport libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

In [76]:
# import the data set of movie ratings 
df = pd.read_csv('../data/ml-latest-small/ratings.csv', na_values = 'Nan')

In [77]:
df.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

In [78]:
# use pivot to make the matrix of movie rates
rates =df.pivot(index='userId',columns = 'movieId')
rates.head()

rating                                                                 \
movieId 1      2      3      4      5      6      7      8      9      10       
userId                                                                          
1          4.0    NaN    4.0    NaN    NaN    4.0    NaN    NaN    NaN    NaN   
2          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5          4.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

         ... timestamp                                                   \
movieId  ...    193565 193567 193571 193573 193579 193581 193583 193585   
userId   ...                                                              
1        ...       NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2        ...       NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3        ...       NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4        ...       NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5        ...       NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

                       
movieId 193587 193609  
userId                 
1          NaN    NaN  
2          NaN    NaN  
3          NaN    NaN  
4          NaN    NaN  
5          NaN    NaN  

[5 rows x 19448 columns]

In [79]:
rates.rating

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
# import the data set of movie names
df_movie = pd.read_csv('../data/ml-latest-small/movies.csv', na_values = 'Nan')

In [81]:
df_movie.title

0                                Toy Story (1995)
1                                  Jumanji (1995)
2                         Grumpier Old Men (1995)
3                        Waiting to Exhale (1995)
4              Father of the Bride Part II (1995)
                          ...                    
9737    Black Butler: Book of the Atlantic (2017)
9738                 No Game No Life: Zero (2017)
9739                                 Flint (2017)
9740          Bungo Stray Dogs: Dead Apple (2018)
9741          Andrew Dice Clay: Dice Rules (1991)
Name: title, Length: 9742, dtype: object

In [82]:
# Split the movie name from movie year and apply it in the matrix
df_movie['year'] = df_movie.title.astype(str).str[-6:]
df_movie['title_new'] = df_movie.title.astype(str).str[:-6]

In [83]:
df_movie

,movieId,title,genres,year,title_new
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,(1995),Toy Story
1,2,Jumanji (1995),Adventure|Children|Fantasy,(1995),Jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,(1995),Grumpier Old Men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,(1995),Waiting to Exhale
4,5,Father of the Bride Part II (1995),Comedy,(1995),Father of the Bride Part II
...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,(2017),Black Butler: Book of the Atlantic
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,(2017),No Game No Life: Zero
9739,193585,Flint (2017),Drama,(2017),Flint
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,(2018),Bungo Stray Dogs: Dead Apple


In [84]:
# Try to zip columns with movie names
#new_columns = dict(zip(df.movieId,df_movie.title))
rates.rename(columns=dict(zip(df_movie["movieId"], df_movie["title_new"])),inplace = True)
rates.head()

rating                                                \
movieId Toy Story  Jumanji  Grumpier Old Men  Waiting to Exhale    
userId                                                             
1              4.0      NaN               4.0                NaN   
2              NaN      NaN               NaN                NaN   
3              NaN      NaN               NaN                NaN   
4              NaN      NaN               NaN                NaN   
5              4.0      NaN               NaN                NaN   

                                                                   \
movieId Father of the Bride Part II  Heat  Sabrina  Tom and Huck    
userId                                                              
1                                NaN   4.0      NaN           NaN   
2                                NaN   NaN      NaN           NaN   
3                                NaN   NaN      NaN           NaN   
4                                NaN   NaN      NaN           NaN   
5                                NaN   NaN      NaN           NaN   

                                  ...           timestamp  \
movieId Sudden Death  GoldenEye   ... Gintama: The Movie    
userId                            ...                       
1                 NaN        NaN  ...                 NaN   
2                 NaN        NaN  ...                 NaN   
3                 NaN        NaN  ...                 NaN   
4                 NaN        NaN  ...                 NaN   
5                 NaN        NaN  ...                 NaN   

                                                                        \
movieId anohana: The Flower We Saw That Day - The Movie  Silver Spoon    
userId                                                                   
1                                                    NaN           NaN   
2                                                    NaN           NaN   
3                                                    NaN           NaN   
4                                                    NaN           NaN   
5                                                    NaN           NaN   

                                                                              \
movieId Love Live! The School Idol Movie  Jon Stewart Has Left the Building    
userId                                                                         
1                                     NaN                                NaN   
2                                     NaN                                NaN   
3                                     NaN                                NaN   
4                                     NaN                                NaN   
5                                     NaN                                NaN   

                                                                           \
movieId Black Butler: Book of the Atlantic  No Game No Life: Zero  Flint    
userId                                                                      
1                                       NaN                    NaN    NaN   
2                                       NaN                    NaN    NaN   
3                                       NaN                    NaN    NaN   
4                                       NaN                    NaN    NaN   
5                                       NaN                    NaN    NaN   

                                                                     
movieId Bungo Stray Dogs: Dead Apple  Andrew Dice Clay: Dice Rules   
userId                                                               
1                                 NaN                           NaN  
2                                 NaN                           NaN  
3                                 NaN                           NaN  
4                                 NaN                           NaN  
5                                 NaN                           NaN  

[5 rows x 194

In [85]:
rate_movies =  rates.rating

In [ ]:
# Deal with the NaN values in the ratings
imputer = SimpleImputer(strategy = 'mean') # add the NaN with the average of movie recommendations 
Rtrue = imputer.fit_transform(df)
Rtrue